<a href="https://colab.research.google.com/github/kaleb36/tuner_mnist/blob/main/tuner_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#installing keras tuner
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.3 MB/s eta 0:00:00


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
import numpy as np

def build_model(hp):
    inputs = keras.Input(shape=(28, 28, 1))
    #tune the number of layers
    for i in range(hp.Int("num_layers", 1, 3)):
        x = layers.Conv2D(
            filters=hp.Int(f"filters_{i}", min_value=32, max_value=128),
            kernel_size=3, activation="relu")(inputs)
        #tune for maxpooling
        if hp.Boolean("maxpooling2d"):
            x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    #tuning for optimizer
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model
#instantiating the model
build_model(keras_tuner.HyperParameters())

In [12]:
tuner = tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    overwrite=True,
    directory="tuner_mnist",
    project_name="mnistv1",
)

In [13]:
#print the summary of the search space
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 1, 'sampling': 'linear'}
maxpooling2d (Boolean)
{'default': False, 'conditions': []}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [14]:
#preparing the dataset
(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10


11490434/11490434 [==============================] - 0s 0us/step


In [17]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
]

#starting the search
tuner.search(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_val, y_val), callbacks=callbacks, verbose=2)

Trial 10 Complete [00h 00m 51s]
val_accuracy: 0.9814999997615814

Best val_accuracy So Far: 0.9848000109195709
Total elapsed time: 00h 06m 40s


In [18]:
#query the results
#get the top 2 best models

models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 78)        780       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 78)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 13182)             0         
                                                                 
 dense (Dense)               (None, 10)                131830    
                                                                 
Total params: 132610 (518.01 KB)
Trainable params: 132610 (518.01 KB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [19]:
#printing the summary of ther search results
tuner.results_summary()

Results summary
Results in tuner_mnist/mnistv1
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
num_layers: 1
filters_0: 78
maxpooling2d: True
optimizer: rmsprop
filters_1: 122
Score: 0.9848000109195709

Trial 02 summary
Hyperparameters:
num_layers: 1
filters_0: 76
maxpooling2d: True
optimizer: adam
Score: 0.9838500022888184
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 309, in run_trial
    self._configure_tensorboard_dir(callbacks, trial, execution)
  File "/usr/local/lib/python3.10/dist-

# Retrain the model
If you want to train the model with the entire dataset, you may retrieve the best hyperparameters and retrain the model by yourself.

In [20]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)

1875/1875 [==============================] - 6s 3ms/step - loss: 0.1799 - accuracy: 0.9468


In [22]:
#Getting the best epoch
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=10,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

#getting the best model
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_all, y_all,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/10
391/391 [==============================] - 3s 6ms/step - loss: 0.3102 - accuracy: 0.9114 - val_loss: 0.1344 - val_accuracy: 0.9657
Epoch 2/10
391/391 [==============================] - 2s 5ms/step - loss: 0.1111 - accuracy: 0.9684 - val_loss: 0.0875 - val_accuracy: 0.9756
Epoch 3/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0777 - accuracy: 0.9777 - val_loss: 0.0752 - val_accuracy: 0.9790
Epoch 4/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0628 - accuracy: 0.9814 - val_loss: 0.0695 - val_accuracy: 0.9802
Epoch 5/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0541 - accuracy: 0.9841 - val_loss: 0.0612 - val_accuracy: 0.9821
Epoch 6/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0476 - accuracy: 0.9862 - val_loss: 0.0680 - val_accuracy: 0.9797
Epoch 7/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0427 - accuracy: 0.9875 - val_loss: 0.0649 - val_accuracy: 0.9809
Epoch 

In [29]:
best_models[3].summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_24 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 13, 13, 32)        0         
 ng2D)                                                           
                                                                 
 flatten_14 (Flatten)        (None, 5408)              0         
                                                                 
 dense_14 (Dense)            (None, 10)                54090     
                                                                 
Total params: 54410 (212.54 KB)
Trainable params: 54410 (212.54 KB)
Non-trainable params: 0 (0.00 Byte)
____________________